<a href="https://colab.research.google.com/github/Poojitha319/Chess-Minimax/blob/main/Chess_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import os
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
df=pd.read_csv('/content/data.csv',encoding='utf-8', quotechar='"', delimiter=',')

In [ ]:
df.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
0,R,N,B,Q,K,B,N,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,R,N,B,Q,K,B,N,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R,N,B,Q,K,B,N,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,R,N,B,Q,K,B,N,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,R,N,B,Q,K,B,N,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.shape

(82600, 193)

In [ ]:
object_columns = df.select_dtypes(include='object').columns
object_columns

Index(['a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1', 'h1', 'a2', 'b2', 'c2', 'd2',
       'e2', 'f2', 'g2', 'h2', 'a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3',
       'a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4', 'h4', 'a5', 'b5', 'c5', 'd5',
       'e5', 'f5', 'g5', 'h5', 'a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6',
       'a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7', 'a8', 'b8', 'c8', 'd8',
       'e8', 'f8', 'g8', 'h8'],
      dtype='object')

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame
# Replace 'good_move' with the actual column name if it's different
categorical_columns = df.select_dtypes(include='object').columns

# Perform one-hot encoding for the specified columns
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Display the encoded DataFrame
print(df_encoded)


In [ ]:
features = list(df.iloc[:, 0:192].columns)

#df.fillna('',inplace=True)

In [ ]:
df.dropna(axis=0,inplace=True)
df.shape

(82599, 193)

In [ ]:
X=df[features]
Y=df['good_move']

In [ ]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name))


In [ ]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [ ]:
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []

  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)

  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)

  batches_X = []
  batches_y = []

  for i in range(0, len(intervals) - 1):
    batches_X.append(df.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(df.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

In [ ]:
batches_X, batches_y = split_into_batches(df)


In [ ]:
linear_est = tf.estimator.LinearRegressor(feature_columns = feature_columns,model_dir='/content/gdrive/MyDrive/Chess_engine')

In [ ]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))
len(input_functions)

1

list

In [ ]:
!pip install chess


In [ ]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)


# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '/content/gdrive/MyDrive/Chess_engine'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)